In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS 
WITH adhoc_dsd AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_silver_dataportal.winlife_mdq_adhoc 
    QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) = 1
), 
adhoc_dc AS (
    SELECT  
        store_id,
        product_id,
        min_qty mdq
    FROM {catalog_name}.udp_wcm_silver_dim.dc_flow_mdq_adhoc
    QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY proc_date DESC) = 1
), 
prev AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist
    WHERE calday = (SELECT MAX(calday) FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist WHERE calday < DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) 
),
mdq AS (
    SELECT      
        a.store_id,
        a.product_id,
        IF(a1.store_id IS NULL, 'DSD', 'DC') replenishment_mode,
        a.assortment_type,
        a4.approval_status AS pog_mdq_approval_status,
        CASE 
            WHEN a4.mdq IS NOT NULL THEN a4.mdq_source
            WHEN a1.store_id IS NULL AND a2.mdq IS NOT NULL THEN 'DSD UPLOAD'
            WHEN a1.store_id IS NOT NULL AND a3.mdq IS NOT NULL THEN 'DC UPLOAD'
            WHEN a.mdq IS NOT NULL THEN 'AUTO MDQ' 
        END mdq_source,
        IFNULL(a4.winare_mdq, IFNULL(IF(a1.store_id IS NULL, a2.mdq, a3.mdq), a.mdq)) mdq,-- WINARE MDQ
        IFNULL(a4.external_report_mdq, IFNULL(IF(a1.store_id IS NULL, a2.mdq, a3.mdq), a.mdq)) external_report_mdq,
        IFNULL(a4.mdq, IFNULL(IF(a1.store_id IS NULL, a2.mdq, a3.mdq), a.mdq)) internal_report_mdq,
        IFNULL(IF(a1.store_id IS NULL, a2.mdq, a3.mdq), a.mdq) old_mdq
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_auto_daily a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc a1 
        USING (store_id, product_id)
    LEFT JOIN adhoc_dsd a2 
        ON a.store_id = a2.store_id AND a.product_id = a2.product_id
    LEFT JOIN adhoc_dc a3 
        ON a.store_id = a3.store_id AND a.product_id = a3.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_store_sku a4 
        ON a.store_id = a4.store_id AND a.product_id = a4.product_id
),
adhoc_win_the_south AS (
    SELECT
        a.store_id,
        a.product_id,
        a.replenishment_mode,
        a.assortment_type,
        a.pog_mdq_approval_status,
        IF(b.store_id IS NOT NULL, 'MDQ - WIN THE SOUTH', a.mdq_source) mdq_source,
        IF(b.store_id IS NOT NULL, b.mdq, a.mdq) mdq,
        a.external_report_mdq,
        IF(b.store_id IS NOT NULL, b.mdq, a.internal_report_mdq) internal_report_mdq,
        a.old_mdq
    FROM mdq a
    LEFT JOIN (
        SELECT b.store_id, a.product_id, a.mdq
        FROM {catalog_name}.udp_wcm_silver_dataportal.d_mdq_sku_adhoc a
        CROSS JOIN (
            SELECT store_id
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
            WHERE region_domain_vn IN ('Miền Nam', 'Miền Tây') AND business_unit = '2000'
        ) b
        WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN start_date AND end_date
    ) b
        ON a.store_id = b.store_id AND a.product_id = b.product_id
)
SELECT DISTINCT
    a.*,
    IF(IFNULL(a.mdq, 0) = 0, NULL, IFNULL(a1.mdq_age, 0) + 1) mdq_age,
    CASE
        WHEN a.mdq IS NULL THEN 'NOT AVAILABLE'
        WHEN a.mdq IS NOT NULL AND a.mdq <> 0 THEN 'AVAILABLE'
        WHEN a.mdq IS NOT NULL AND a.mdq = 0 THEN '0'
    END mdq_available
FROM adhoc_win_the_south a
LEFT JOIN prev a1 USING (store_id, product_id)
;

""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist
(
  calday DATE,
  store_id STRING,
  product_id STRING,
  replenishment_mode STRING,
  assortment_type STRING,
  pog_mdq_approval_status STRING,
  mdq_source STRING,
  mdq FLOAT,
  mdq_age BIGINT,
  mdq_available STRING,
  external_report_mdq FLOAT,
  internal_report_mdq FLOAT,
  old_mdq FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist
WHERE CALDAY = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday, 
    store_id,
    product_id,
    replenishment_mode,
    assortment_type,
    pog_mdq_approval_status,
    mdq_source,
    mdq,
    mdq_age,
    mdq_available,
    external_report_mdq,
    internal_report_mdq,
    old_mdq
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily;
""")